In [1]:
# import sys
# sys.path.insert(0,'../')
from file_tools import *
from request_tools import *
from parse_tools import *
from load_tools import *
import time
from tqdm import tqdm
import argparse
import warnings
import os
from pandas.errors import PerformanceWarning
pd.options.mode.use_inf_as_na = True
warnings.filterwarnings('ignore', category=PerformanceWarning)

# parser = argparse.ArgumentParser(description='Normalize basketball-reference.com data')
# parser.add_argument('-d', '--debug', action='store_true', help='Debug mode')
# parser.add_argument('-s', '--sourcedir', type=str, default='../data-parsed/', help='Source directory')
# parser.add_argument('-t', '--targetdir', type=str, default='../data-indexed/', help='Target directory')
# parser.add_argument('-m', '--mode', type=str, default='all', help='')

In [2]:
def __load_season_stats_from_dir__(directory):
    df = load_concat_df_from_dir(directory,index_col=[0,1],header=[0,1])
    return df

def __load_season_facts_from_dir__(directory):
    df = load_concat_df_from_dir(directory,index_col=[0,1],header=[0])
    return df

def __get_opp_idx__(IDX_DF,TM_IDX):
    return IDX_DF.loc[TM_IDX,'Opp']

def __get_team_opp_stats_row_df__(STATS_DF,IDX_DF,TM_IDX):
    OPP_IDX = __get_opp_idx__(IDX_DF,TM_IDX)
    TM_STATS_DF = STATS_DF.loc[TM_IDX]
    OPP_STATS_DF = STATS_DF.loc[OPP_IDX]
    TM_OPP_STATS_DF = pd.concat([TM_STATS_DF,OPP_STATS_DF],keys=['Team_Stats','Opponent_Stats']).to_frame().T
    TM_OPP_STATS_DF.index = pd.MultiIndex.from_tuples([TM_IDX],names=['index','Team_id'])
    return TM_OPP_STATS_DF

def get_team_opp_stats_df(STATS_DF,IDX_DF):
    TM_OPP_STATS_DF_LIST = []
    for TM_IDX in STATS_DF.index:
        TM_OPP_STATS_DF_LIST.append(__get_team_opp_stats_row_df__(STATS_DF,IDX_DF,TM_IDX))
    TM_OPP_STATS_DF = pd.concat(TM_OPP_STATS_DF_LIST,axis=0)
    return TM_OPP_STATS_DF

def __reindex_stats__(STATS_DF,IDX_DF):
    STATS_DF_SRC = STATS_DF.copy()
    STATS_DF_REIDX = STATS_DF.copy()
    STATS_DF_REIDX.loc[(-1,'-1'),:] = pd.NA
    IDX = pd.MultiIndex.from_tuples(IDX_DF, names=['index', 'Team_id'])
    STATS_DF_REIDX = STATS_DF_REIDX.loc[IDX].reset_index(col_level=1, col_fill='Source')
    STATS_DF_REIDX.index = STATS_DF_SRC.index
    return STATS_DF_REIDX


In [42]:
# Reindex all of the stats dataframes from a given season
LG_FACTS_DIR = './00-data-facts/'
IDX_DIR = './05-data-indexed/'
SRC_DIR = './04-data-normalized/'
TGT_DIR = './06-data-normalized-reidx-test/'

LG_SS_HTML_DICT = load_league_seasons_dict(LG_FACTS_DIR)
# TQDM_LG_SS_HTML_DICT_KEYS = tqdm(list(LG_SS_HTML_DICT.keys()),ncols=150)
# for LG_SS_HTML in TQDM_LG_SS_HTML_DICT_KEYS:
LG_SS_HTML = '/leagues/NBA_1980.html'
LG_SS_DIR  = parse_league_id(LG_SS_HTML)['body']

IDX_DICT = load_index_dict(f'{IDX_DIR}/{LG_SS_DIR}')
TM_OPP_MN_IDX = IDX_DICT['team_opp_main']

DF_REIDX = {}
for NORM_TYPE_DIR in ['norm_standard','unnorm']:#,'norm_minmax','norm_robust','norm_ranking','unnorm']:
    for TGL_TYPE_DIR in get_all_folders(f'{SRC_DIR}/{LG_SS_DIR}'):
        HM_AGG_TYPE_DIR,AW_AGG_TYPE_DIR = [],[]
        for AGG_TYPE_DIR in get_all_folders(f'{SRC_DIR}/{LG_SS_DIR}/{TGL_TYPE_DIR}'):
            if 'hm' in AGG_TYPE_DIR:
                HM_AGG_TYPE_DIR.append(AGG_TYPE_DIR)
            elif 'aw' in AGG_TYPE_DIR:
                AW_AGG_TYPE_DIR.append(AGG_TYPE_DIR)
            elif 'facts' in AGG_TYPE_DIR:
                TGL_FACTS_DF_REIDX = __load_season_facts_from_dir__('/'.join([SRC_DIR,LG_SS_DIR,TGL_TYPE_DIR,AGG_TYPE_DIR,'facts']))
                # TGL_REIDX_FNAME = f'{TGL_TYPE_DIR}_{AGG_TYPE_DIR}'
                make_directory('/'.join([TGT_DIR,LG_SS_DIR]))
                TGL_REIDX_FNAME = f'{AGG_TYPE_DIR}_{TGL_TYPE_DIR}.csv'
                TGL_FACTS_DF_REIDX.to_csv('/'.join([TGT_DIR,LG_SS_DIR,TGL_REIDX_FNAME]),index=True)
                # DF_REIDX[TGL_REIDX_FNAME] = TGL_FACTS_DF
            else:
                TGL_STATS_AGG_NORM_DF = __load_season_stats_from_dir__('/'.join([SRC_DIR,LG_SS_DIR,TGL_TYPE_DIR,AGG_TYPE_DIR,NORM_TYPE_DIR]))
                TGL_STATS_AGG_NORM_DF_REIDX = __reindex_stats__(TGL_STATS_AGG_NORM_DF,TM_OPP_MN_IDX['Team_Prev_Gm_01'])
                make_directory('/'.join([TGT_DIR,LG_SS_DIR]))
                TGL_REIDX_FNAME = f'{NORM_TYPE_DIR}_{TGL_TYPE_DIR}_{AGG_TYPE_DIR}.csv'
                TGL_STATS_AGG_NORM_DF_REIDX.to_csv('/'.join([TGT_DIR,LG_SS_DIR,TGL_REIDX_FNAME]),index=True)
                # DF_REIDX[TGL_REIDX_FNAME] = TGL_STATS_AGG_NORM_DF_REIDX
        for TGL_HM_TYPE, TGL_AW_TYPE in list(zip(sorted(HM_AGG_TYPE_DIR),sorted(AW_AGG_TYPE_DIR))):
            TGL_HM_STATS_AGG_NORM_DF = __load_season_stats_from_dir__('/'.join([SRC_DIR,LG_SS_DIR,TGL_TYPE_DIR,TGL_HM_TYPE,NORM_TYPE_DIR]))
            TGL_AW_STATS_AGG_NORM_DF = __load_season_stats_from_dir__('/'.join([SRC_DIR,LG_SS_DIR,TGL_TYPE_DIR,TGL_AW_TYPE,NORM_TYPE_DIR]))
            TGL_VEN_STATS_AGG_NORM_DF = pd.concat([TGL_HM_STATS_AGG_NORM_DF,TGL_AW_STATS_AGG_NORM_DF],axis=0).sort_index()
            TGL_VEN_STATS_AGG_NORM_DF_REIDX = __reindex_stats__(TGL_VEN_STATS_AGG_NORM_DF,TM_OPP_MN_IDX['Team_Ven_Prev_Gm_01'])
            TGL_VEN_REIDX_FNAME = f'{NORM_TYPE_DIR}_{TGL_TYPE_DIR}_{TGL_HM_TYPE.replace("hm","ven")}.csv'
            make_directory('/'.join([TGT_DIR,LG_SS_DIR]))
            TGL_VEN_STATS_AGG_NORM_DF_REIDX.to_csv('/'.join([TGT_DIR,LG_SS_DIR,TGL_VEN_REIDX_FNAME]),index=True)
            # DF_REIDX[TGL_VEN_REIDX_FNAME] = TGL_VEN_STATS_AGG_NORM_DF_REIDX
            
# DF_REIDX.keys()             

ValueError: Length mismatch: Expected axis has 1804 elements, new values have 1802 elements

In [50]:
TGL_HM_STATS_AGG_NORM_DF

Result                          Team             
                                 W/L        Tm       Opp        FG       FGA   
index Team_id                                                                  
0     /teams/ATL/1980.html  0.683130  1.277066 -0.167553  1.664103  0.610433  \
      /teams/BOS/1980.html  0.683130  0.523548 -0.010695  0.498171  0.735942   
4     /teams/CHI/1980.html  0.683130  0.738839 -0.167553 -0.434575 -2.025268   
1     /teams/CLE/1980.html -1.463850  0.846485  2.577462  0.731357  0.735942   
0     /teams/DEN/1980.html -1.463850 -1.414070  1.479456 -2.300066  1.237980   
...                              ...       ...       ...       ...       ...   
81    /teams/SAS/1980.html  0.050018  2.449069  2.513360  1.960877  0.897616   
79    /teams/SDC/1980.html -0.400143 -0.055333  0.735245 -0.030867  0.081428   
81    /teams/SEA/1980.html  0.950339 -0.214511 -1.195430 -0.184078  0.968310   
79    /teams/UTA/1980.html -1.450518 -1.339370 -0.111727 -1.397981 -2.322150   
80    /teams/WSB/1980.html -0.400143 -0.904283 -0.264287 -0.054438  1.154683   

                                                                               
                                 FG%        3P       3PA       3P%        FT   
index Team_id                                                                  
0     /teams/ATL/1980.html  0.908798 -0.510310 -0.817432       NaN -0.355518  \
      /teams/BOS/1980.html -0.170400  0.612372  0.999083  0.023405  0.008268   
4     /teams/CHI/1980.html  1.438930 -0.510310 -0.211927 -1.048264  2.009088   
1     /teams/CLE/1980.html  0.018933  0.612372  0.999083  0.023405  0.190161   
0     /teams/DEN/1980.html -2.802128  0.612372  0.393578  0.560849  1.099624   
...                              ...       ...       ...       ...       ...   
81    /teams/SAS/1980.html  1.502868 -0.470101 -0.631058 -0.126321  1.479993   
79    /teams/SDC/1980.html -0.158934  2.525288  2.362107  1.157222 -0.687877   
81    /teams/SEA/1980.html -1.408898 -0.301347 -0.374958  0.570051 -0.021654   
79    /teams/UTA/1980.html  0.939301 -0.174781 -0.374958  0.516167 -0.116828   
80    /teams/WSB/1980.html -1.495601 -0.048216 -0.099159  1.503337 -1.692499   

                            ...  Opponent                                 
                            ...        FT       FTA       FT%       ORB   
index Team_id               ...                                           
0     /teams/ATL/1980.html  ...  0.555778  0.176802  0.859409       NaN  \
      /teams/BOS/1980.html  ...  0.421414  0.399066  0.125349 -0.419019   
4     /teams/CHI/1980.html  ...  0.824505  1.399258 -0.697988 -0.419019   
1     /teams/CLE/1980.html  ...  2.033780  1.732655  0.750292 -1.528188   
0     /teams/DEN/1980.html  ... -0.116041  0.287934 -0.797185  1.429595   
...                         ...       ...       ...       ...       ...   
81    /teams/SAS/1980.html  ... -0.134222 -0.282618  0.520099 -0.924279   
79    /teams/SDC/1980.html  ... -0.441469 -0.622073  0.466303 -0.935604   
81    /teams/SEA/1980.html  ... -0.374676 -0.403070 -0.043988  0.387847   
79    /teams/UTA/1980.html  ... -0.267808 -0.512572  0.498581 -1.333263   
80    /teams/WSB/1980.html  ... -0.828868 -0.742525 -0.465132 -0.527878   

                                                                               
                                 TRB       AST       STL       BLK       TOV   
index Team_id                                                                  
0     /teams/ATL/1980.html  0.028936 -0.720531       NaN -0.845154  0.941338  \
      /teams/BOS/1980.html -1.244244 -0.720531 -0.809207 -0.845154  0.095191   
4     /teams/CHI/1980.html -0.607654 -0.183186  0.754345 -0.362209  2.422094   
1     /teams/CLE/1980.html  0.028936  2.234867 -0.809207 -0.845154  0.095191   
0     /teams/DEN/1980.html  2.150903  0.354159  1.014937  1.569572 -0.327882   
...                              ...       ...       ...       ...       ...   
81    /teams/SAS/

In [43]:
TGL_STATS_AGG_NORM_DF

Result                          Team             
                                 W/L        Tm       Opp        FG       FGA   
index Team_id                                                                  
0     /teams/ATL/1980.html  1.095445  2.308228  0.121858  2.260452  0.796271  \
      /teams/BOS/1980.html  1.095445  1.481560  0.328079  1.160232  0.912284   
      /teams/CHI/1980.html  1.095445  0.064416 -0.703025  0.060012 -0.827911   
      /teams/CLE/1980.html -0.912871 -0.762252 -0.084363  0.280056  1.376336   
      /teams/DEN/1980.html -0.912871 -0.644157  2.287176 -1.480296  1.376336   
...                              ...       ...       ...       ...       ...   
81    /teams/SAS/1980.html  0.000000  2.434854  2.317122  1.740927  1.096253   
      /teams/SDC/1980.html -0.492441 -0.430408  0.528228 -0.344645  0.165498   
      /teams/SEA/1980.html  1.231103 -0.202490 -1.222661 -0.156189  0.601465   
      /teams/UTA/1980.html -1.395250 -1.691361 -0.212846 -1.236667 -2.176961   
      /teams/WSB/1980.html -0.164147 -0.569527  0.045037 -0.030553  1.231195   

                                                                               
                                 FG%        3P       3PA       3P%        FT   
index Team_id                                                                  
0     /teams/ATL/1980.html  1.469087 -0.461084 -0.750000       NaN -0.118040  \
      /teams/BOS/1980.html  0.326261  0.666010  0.500000  0.721325  0.281481   
      /teams/CHI/1980.html  0.827501 -0.461084 -0.333333 -0.733181  0.081720   
      /teams/CLE/1980.html -0.856664  0.666010  0.083333  1.450762 -1.915887   
      /teams/DEN/1980.html -2.460629  0.666010  0.083333  1.450762  1.480046   
...                              ...       ...       ...       ...       ...   
81    /teams/SAS/1980.html  0.978318 -0.284017 -0.233653 -0.425523  1.632185   
      /teams/SDC/1980.html -0.676334  2.743296  2.756037  1.186905 -0.868150   
      /teams/SEA/1980.html -0.863100 -0.114487 -0.393339  1.360981 -0.081331   
      /teams/UTA/1980.html  0.926610 -0.114487 -0.473183  1.254884 -1.008029   
      /teams/WSB/1980.html -1.470475  0.224572  0.143550  1.364447 -1.118766   

                            ...  Opponent                                 
                            ...        FT       FTA       FT%       ORB   
index Team_id               ...                                           
0     /teams/ATL/1980.html  ...  1.643115  1.078222  0.641664       NaN  \
      /teams/BOS/1980.html  ...  1.421345  1.414689 -0.043788 -0.991729   
      /teams/CHI/1980.html  ... -1.018127 -0.435877 -1.062703  1.223836   
      /teams/CLE/1980.html  ...  0.312494  0.237056  0.039578  2.173363   
      /teams/DEN/1980.html  ...  0.534264  1.246456 -0.905234  0.590817   
...                         ...       ...       ...       ...       ...   
81    /teams/SAS/1980.html  ... -0.094598  0.009055 -0.561806 -0.151030   
      /teams/SDC/1980.html  ... -0.956268 -1.075511  0.402585 -0.028808   
      /teams/SEA/1980.html  ... -0.759106 -0.721367 -0.365971 -0.272074   
      /teams/UTA/1980.html  ... -0.306364 -0.394891  0.008454 -1.439433   
      /teams/WSB/1980.html  ... -0.350178 -0.516628  1.118181 -0.864874   

                                                                               
                                 TRB       AST       STL       BLK       TOV   
index Team_id                                                                  
0     /teams/ATL/1980.html -0.415645 -0.603623       NaN -0.643041  0.955654  \
      /teams/BOS/1980.html -1.361596 -0.603623 -0.969846 -0.643041  0.175528   
      /teams/CHI/1980.html  1.318598  0.561263  0.057050 -0.183726  0.760623   
      /teams/CLE/1980.html  2.264549  0.095309  1.340670  1.653535  1.345717   
      /teams/DEN/1980.html  1.160940  0.328286  0.827222  1.653535 -0.214535   
...                              ...       ...       ...       ...       ...   
81    /teams/SAS/

In [45]:
TGL_STATS_AGG_NORM_DF.index.difference(TGL_VEN_STATS_AGG_NORM_DF.index)

MultiIndex([(80, '/teams/MIL/1980.html'),
            (81, '/teams/PHO/1980.html')],
           names=['index', 'Team_id'])

In [46]:
TM_OPP_MN_IDX.loc[TGL_STATS_AGG_NORM_DF.index.difference(TGL_VEN_STATS_AGG_NORM_DF.index)]

,,Opp_Curr_Gm,Opp_Prev_Gm_01,Opp_Prev_Gm_Opp_01,Opp_Prev_H2H_Gm_01,Opp_Ven_Prev_Gm_01,Team_Curr_Gm,Team_Prev_Gm_01,Team_Prev_Gm_Opp_01,Team_Prev_H2H_Gm_01,Team_Ven_Prev_Gm_01
index,Team_id,,,,,,,,,,
80,/teams/MIL/1980.html,"(80, /teams/KCK/1980.html)","(79, /teams/KCK/1980.html)","(79, /teams/GSW/1980.html)","(73, /teams/KCK/1980.html)","(75, /teams/KCK/1980.html)","(80, /teams/MIL/1980.html)","(79, /teams/MIL/1980.html)","(81, /teams/DEN/1980.html)","(74, /teams/MIL/1980.html)","(78, /teams/MIL/1980.html)"
81,/teams/PHO/1980.html,"(81, /teams/SDC/1980.html)","(80, /teams/SDC/1980.html)","(80, /teams/LAL/1980.html)","(74, /teams/SDC/1980.html)","(80, /teams/SDC/1980.html)","(81, /teams/PHO/1980.html)","(80, /teams/PHO/1980.html)","(80, /teams/POR/1980.html)","(72, /teams/PHO/1980.html)","(80, /teams/PHO/1980.html)"


In [51]:
TM_OPP_MN_IDX['Team_Ven_Prev_Gm_01'].loc[TGL_STATS_AGG_NORM_DF.index.difference(TGL_VEN_STATS_AGG_NORM_DF.index)]

index  Team_id             
80     /teams/MIL/1980.html    (78, /teams/MIL/1980.html)
81     /teams/PHO/1980.html    (80, /teams/PHO/1980.html)
Name: Team_Ven_Prev_Gm_01, dtype: object

In [41]:
TM_OPP_MN_IDX['Team_Prev_Gm_01']

index  Team_id             
0      /teams/ATL/1980.html                      (-1, -1)
       /teams/BOS/1980.html                      (-1, -1)
       /teams/CHI/1980.html                      (-1, -1)
       /teams/CLE/1980.html                      (-1, -1)
       /teams/DEN/1980.html                      (-1, -1)
                                          ...            
81     /teams/SAS/1980.html    (80, /teams/SAS/1980.html)
       /teams/SDC/1980.html    (80, /teams/SDC/1980.html)
       /teams/SEA/1980.html    (80, /teams/SEA/1980.html)
       /teams/UTA/1980.html    (80, /teams/UTA/1980.html)
       /teams/WSB/1980.html    (80, /teams/WSB/1980.html)
Name: Team_Prev_Gm_01, Length: 1804, dtype: object

In [34]:
__reindex_stats__(TGL_STATS_AGG_NORM_DF,TM_OPP_MN_IDX['Team_Prev_Gm_01'])

KeyError: "[(67, '/teams/BOS/2020.html'), (67, '/teams/BRK/2020.html'), (67, '/teams/DAL/2020.html'), (67, '/teams/DEN/2020.html'), (67, '/teams/HOU/2020.html'), (67, '/teams/IND/2020.html'), (67, '/teams/LAC/2020.html'), (67, '/teams/LAL/2020.html'), (67, '/teams/MEM/2020.html'), (67, '/teams/MIA/2020.html'), (67, '/teams/MIL/2020.html'), (67, '/teams/NOP/2020.html'), (67, '/teams/OKC/2020.html'), (67, '/teams/ORL/2020.html'), (67, '/teams/PHI/2020.html'), (67, '/teams/PHO/2020.html'), (67, '/teams/POR/2020.html'), (67, '/teams/SAC/2020.html'), (67, '/teams/SAS/2020.html'), (67, '/teams/TOR/2020.html'), (67, '/teams/UTA/2020.html'), (67, '/teams/WAS/2020.html'), (68, '/teams/BOS/2020.html'), (68, '/teams/BRK/2020.html'), (68, '/teams/DAL/2020.html'), (68, '/teams/DEN/2020.html'), (68, '/teams/HOU/2020.html'), (68, '/teams/IND/2020.html'), (68, '/teams/LAC/2020.html'), (68, '/teams/LAL/2020.html'), (68, '/teams/MEM/2020.html'), (68, '/teams/MIA/2020.html'), (68, '/teams/MIL/2020.html')

In [30]:
TM_OPP_MN_IDX.tail(50)

Opp_Curr_Gm   
index Team_id                                            
70    /teams/IND/2020.html  (70, /teams/MIA/2020.html)  \
      /teams/LAC/2020.html  (71, /teams/DEN/2020.html)   
      /teams/LAL/2020.html  (71, /teams/SAC/2020.html)   
      /teams/MEM/2020.html  (68, /teams/TOR/2020.html)   
      /teams/MIA/2020.html  (70, /teams/IND/2020.html)   
      /teams/MIL/2020.html  (69, /teams/TOR/2020.html)   
      /teams/NOP/2020.html  (70, /teams/SAC/2020.html)   
      /teams/OKC/2020.html  (71, /teams/MIA/2020.html)   
      /teams/ORL/2020.html  (69, /teams/BOS/2020.html)   
      /teams/PHI/2020.html  (71, /teams/PHO/2020.html)   
      /teams/PHO/2020.html  (69, /teams/OKC/2020.html)   
      /teams/POR/2020.html  (68, /teams/LAC/2020.html)   
      /teams/SAC/2020.html  (70, /teams/NOP/2020.html)   
      /teams/SAS/2020.html  (71, /teams/UTA/2020.html)   
      /teams/TOR/2020.html  (71, /teams/PHI/2020.html)   
      /teams/UTA/2020.html  (72, /teams/DAL/2020.html)   
      /teams/WAS/2020.html  (71, /teams/MIL/2020.html)   
71    /teams/BOS/2020.html  (71, /teams/WAS/2020.html)   
      /teams/BRK/2020.html  (73, /teams/POR/2020.html)   
      /teams/DAL/2020.html  (69, /teams/MIL/2020.html)   
      /teams/DEN/2020.html  (70, /teams/LAC/2020.html)   
      /teams/HOU/2020.html  (72, /teams/PHI/2020.html)   
      /teams/IND/2020.html  (70, /teams/HOU/2020.html)   
      /teams/LAC/2020.html  (71, /teams/OKC/2020.html)   
      /teams/MEM/2020.html  (70, /teams/BOS/2020.html)   
      /teams/MIA/2020.html  (70, /teams/OKC/2020.html)   
      /teams/MIL/2020.html  (70, /teams/WAS/2020.html)   
      /teams/NOP/2020.html  (72, /teams/ORL/2020.html)   
      /teams/OKC/2020.html  (71, /teams/LAC/2020.html)   
      /teams/ORL/2020.html  (70, /teams/BRK/2020.html)   
      /teams/PHI/2020.html  (70, /teams/TOR/2020.html)   
      /teams/PHO/2020.html  (70, /teams/PHI/2020.html)   
      /teams/POR/2020.html  (69, /teams/PHI/2020.html)   
      /teams/SAC/2020.html  (70, /teams/LAL/2020.html)   
      /teams/TOR/2020.html  (72, /teams/DEN/2020.html)   
      /teams/UTA/2020.html  (70, /teams/SAS/2020.html)   
      /teams/WAS/2020.html  (71, /teams/BOS/2020.html)   
72    /teams/DAL/2020.html  (70, /teams/UTA/2020.html)   
      /teams/DEN/2020.html  (71, /teams/TOR/2020.html)   
      /teams/IND/2020.html  (72, /teams/MIA/2020.html)   
      /teams/MEM/2020.html  (72, /teams/MIL/2020.html)   
      /teams/MIA/2020.html  (72, /teams/IND/2020.html)   
      /teams/MIL/2020.html  (72, /teams/MEM/2020.html)   
      /teams/ORL/2020.html  (71, /teams/NOP/2020.html)   
      /teams/PHI/2020.html  (71, /teams/HOU/2020.html)   
      /teams/PHO/2020.html  (74, /teams/DAL/2020.html)   
      /teams/POR/2020.html  (73, /teams/DAL/2020.html)   
73    /teams/DAL/2020.html  (72, /teams/POR/2020.html)   
      /teams/POR/2020.html  (71, /teams/BRK/2020.html)   
74    /teams/DAL/2020.html  (72, /teams/PHO/2020.html)   

                                        Opp_Prev_Gm_01   
index Team_id                                            
70    /teams/IND/2020.html  (69, /teams/MIA/2020.html)  \
      /teams/LAC/2020.html  (70, /teams/DEN/2020.html)   
      /teams/LAL/2020.html  (70, /teams/SAC/2020.html)   
      /teams/MEM/2020.html  (67, /teams/TOR/2020.html)   
      /teams/MIA/2020.html  (69, /teams/IND/2020.html)   
      /teams/MIL/2020.html  (68, /teams/TOR/2020.html)   
      /teams/NOP/2020.html  (69, /teams/SAC/2020.html)   
      /teams/OKC/2020.html  (70, /teams/MIA/2020.html)   
      /teams/ORL/2020.html  (68, /teams/BOS/2020.html)   
      /teams/PHI/2020.html  (70, /teams/PHO/2020.html)   
      /teams/PHO/2020.html  (68, /teams/OKC/2020.html)   
      /teams/POR/2020.html  (67, /teams/LAC/2020.html)   
      /teams/SAC/2020.html  (69, /teams/NOP/2020.html)   
      /teams/SAS/2020.html  (70, /teams/UTA/2020.html)   
      /teams/TOR/2020.html  (70, /teams/PHI/2020.html)   
      /teams/UTA/2020.html  (71, /teams/

In [18]:
DF_REIDX['norm_standard_tgl_basic_stats_roll_04_avg']

Source                          Result             
                            index               Team_id       W/L        Tm   
index Team_id                                                                 
0     /teams/ATL/2023.html     -1                    -1       NaN       NaN  \
      /teams/BOS/2023.html     -1                    -1       NaN       NaN   
      /teams/BRK/2023.html     -1                    -1       NaN       NaN   
      /teams/CHI/2023.html     -1                    -1       NaN       NaN   
      /teams/CHO/2023.html     -1                    -1       NaN       NaN   
...                           ...                   ...       ...       ...   
81    /teams/SAC/2023.html     80  /teams/SAC/2023.html -1.074172  0.181064   
      /teams/SAS/2023.html     80  /teams/SAS/2023.html  0.000000  1.172291   
      /teams/TOR/2023.html     80  /teams/TOR/2023.html  0.000000 -0.929110   
      /teams/UTA/2023.html     80  /teams/UTA/2023.html -1.074172 -0.294725   
      /teams/WAS/2023.html     80  /teams/WAS/2023.html -1.074172  0.022468   

                                          Team                                 
                                 Opp        FG       FGA       FG%        3P   
index Team_id                                                                  
0     /teams/ATL/2023.html       NaN       NaN       NaN       NaN       NaN  \
      /teams/BOS/2023.html       NaN       NaN       NaN       NaN       NaN   
      /teams/BRK/2023.html       NaN       NaN       NaN       NaN       NaN   
      /teams/CHI/2023.html       NaN       NaN       NaN       NaN       NaN   
      /teams/CHO/2023.html       NaN       NaN       NaN       NaN       NaN   
...                              ...       ...       ...       ...       ...   
81    /teams/SAC/2023.html  0.757444  0.609221  2.426996 -1.478274  0.036941   
      /teams/SAS/2023.html  2.298009  1.082709  1.872137 -0.465811 -0.184704   
      /teams/TOR/2023.html -1.591917  0.230431  1.009022 -0.854314 -1.403754   
      /teams/UTA/2023.html  0.256761 -0.053662  0.330861 -0.383401 -1.292931   
      /teams/WAS/2023.html  1.258128  0.041036 -0.162348  0.158148 -0.960463   

                                      ...  Opponent                       
                                 3PA  ...        FT       FTA       FT%   
index Team_id                         ...                                 
0     /teams/ATL/2023.html       NaN  ...       NaN       NaN       NaN  \
      /teams/BOS/2023.html       NaN  ...       NaN       NaN       NaN   
      /teams/BRK/2023.html       NaN  ...       NaN       NaN       NaN   
      /teams/CHI/2023.html       NaN  ...       NaN       NaN       NaN   
      /teams/CHO/2023.html       NaN  ...       NaN       NaN       NaN   
...                              ...  ...       ...       ...       ...   
81    /teams/SAC/2023.html  1.192252  ...  1.125293  0.508689  1.244818   
      /teams/SAS/2023.html -0.098607  ...  0.319380  0.441164 -0.326062   
      /teams/TOR/2023.html -0.797823  ... -1.740174 -1.314489 -1.323523   
      /teams/UTA/2023.html -0.475108  ...  1.841660  1.926716 -0.359793   
      /teams/WAS/2023.html -0.797823  ...  0.140289  0.576214 -1.737927   

                                                                               
                                 ORB       TRB       AST       STL       BLK   
index Team_id                                                                  
0     /teams/ATL/2023.html       NaN       NaN       NaN       NaN       NaN  \
      /teams/BOS/2023.html       NaN       NaN       NaN       NaN       NaN   
      /teams/BRK/2023.html       NaN       NaN       NaN       NaN       NaN   
      /teams/CHI/2023.html       NaN       NaN       NaN       NaN       NaN   
      /teams/CHO/2023.html       NaN       NaN       NaN       NaN       NaN   
...                              ...       ...       ...       ...       ...   
81    /teams/SAC/2023.html -

In [14]:
os.path.isdir('/'.join([SRC_DIR,LG_SS_DIR,TGL_TYPE_DIR,AGG_TYPE_DIR,NORM_TYPE_DIR,'']))
'/'.join([SRC_DIR,LG_SS_DIR,TGL_TYPE_DIR,AGG_TYPE_DIR,NORM_TYPE_DIR,''])

'./data-normalized///leagues/NBA_2023/tgl_basic/facts_gm_results/norm_standard/'

In [ ]:
SRC_DIR,LG_SS_DIR,TGL_TYPE_DIR,AGG_TYPE_DIR,NORM_TYPE_DIR

In [5]:
os.path.join(SRC_DIR,LG_SS_DIR)

'/leagues/NBA_2023'